# Deep Q-Learning 

In [2]:
import gym

env = gym.make('BreakoutDeterministic-v4')

state = env.reset()
env.render()
print(len(state))
is_done = False

while not is_done:
    state, reward, is_done, _  = env.step(env.action_space.sample())
    env.render()
env.close()

210


## Preprocessing

In [3]:
def toGrayScale(img):
    return np.mean(img, axis=2).astype(np.uint8) #storing as uint8 to consume less memory

#210x160 -> 105x80
def downSample(img):
    return img[::2, ::2]

In [4]:
def preProcess(img):
    return toGrayScale(downSample(img))

In [5]:
#the input will be a stack of 4 frames to the model get the temporal features 
#clipping reward to limit the scale erros of the derivatives
def transform_reward(reward):
    return np.sign(reward) #return -1 if reward is negativ or 1 if it is positive

In [6]:
class RingBuf:
    def __init__(self, size):
        # Pro-tip: when implementing a ring buffer, always allocate one extra element,
        # this way, self.start == self.end always means the buffer is EMPTY, whereas
        # if you allocate exactly the right number of elements, it could also mean
        # the buffer is full. This greatly simplifies the rest of the code.
        self.data = [None] * (size + 1)
        self.start = 0
        self.end = 0
        
    def append(self, element):
        self.data[self.end] = element
        self.end = (self.end + 1) % len(self.data)
        # end == start and yet we just added one element. This means the buffer has one
        # too many element. Remove the first element by incrementing start.
        if self.end == self.start:
            self.start = (self.start + 1) % len(self.data)
        
    def __getitem__(self, idx):
        return self.data[(self.start + idx) % len(self.data)]
    
    def __len__(self):
        if self.end < self.start:
            return self.end + len(self.data) - self.start
        else:
            return self.end - self.start
        
    def __iter__(self):
        for i in range(len(self)):
            yield self[i]

In [45]:
#Creating the ring buffer and populting it with a random policy
import random
import numpy as np
buffer = RingBuf(500)

c = 0
while(c < 250):
    state = env.reset()
    env.render()
    is_done = False
    while(not is_done):
        action = env.action_space.sample()
        state, reward, is_done, _ = env.step(action)
        buffer.append([preProcess(state), transform_reward(reward), action])
        env.render()
        c = c + 1
env.close()
sample = random.sample(buffer.data, k=3)
print(sample)

[[array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 0.0, 1], None, [array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 0.0, 0]]


In [44]:
from PIL import Image
#len(sample[0][0])
img = Image.fromarray(sample[0][0], mode='RGB')
img.show()

## Building the model 

In [48]:
import keras
#from keras.models import Model
#from keras.layers import Dense

shape = (105, 80, 4)
n_actions = 4

frames_input = keras.layers.Input(shape, name='frames')
actions_input = keras.layers.Input((n_actions,), name='mask')

normalized = keras.layers.lambda(lambda x: x/255.0)(frames_input)

conv_1 = keras.layers.convolutional.Convolution2D(16, 8, 8, subsample=(4, 4), activaion='relu')

AttributeError: 'Discrete' object has no attribute 'length'